## 9.2 ROOM - relacje

Aplikacja będzie zawierała tylko pojedynczą aktywność, jednak skupimy się tylko na bazie `ROOM` i kilku możliwym relacjom pomiędzy tabelami. Poprawność samej bazy zweryfikujemy za pomocą `App Inspector` - nie będziemy przygotowywać layotu ani przesyłać danych z bazy, skupimy się tylko na samym `ROOM`.

Sama baza będzie składała się z kilku tabel reprezentujących wydział. Rozpocznijmy od dodanie odpowiednich zależności do aplikacji

In [ ]:
    // ROOM
    implementation "androidx.room:room-runtime:2.4.3"
    annotationProcessor "androidx.room:room-compiler:2.4.3"

    // ViewModel
    implementation 'androidx.lifecycle:lifecycle-viewmodel:2.5.1'
    // LiveData
    implementation "androidx.lifecycle:lifecycle-livedata:2.5.1"

### **Relacja 1-1**

W relacji 1-1 jednemu elementowi jednej tabeli odpowiada dokładnie jeden element innej tabeli. Rozpocznijmy od dodania `@Entity` reprezentującego dziekana.

In [ ]:
@Entity
data class Dean (

Klasa będzie posiadać dwa pola - `deanName` oraz `facultyName` - pierwsze wykorzystamy jako klucz główny

In [ ]:
@Entity
data class Dean (
    @PrimaryKey(autoGenerate = false)
    val deanName: String,
    val facultyName: String
)

Czyli każdy dziekan posiada swoją nazwę oraz wydział do którego jest przypisany.

Następnie dodajmy `@Entity` opisujący sam wydział.

In [ ]:
@Entity
data class Faculty (
    @PrimaryKey(autoGenerate = false)
    val facultyName: String
)

Klasa `Faculty` posiada jedno pole reprezentujące nazwę wydziału.

Dzięki zastosowaniu adnotacji `@Entity` tabele `Dean` oraz `Faculty` zostaną utworzone w bazie danych. Następnie ustalimy relację pomiędzy tymi dwiema tabelami. Wydział może posiadać tylko jednego dziakana oraz dziekan może zarządzać tylko jednym wydziałem, mamy tutaj relację **1-1** - jest to najprostsza relacja.

Taką relację musimy opisać w odpowiedniej klasie - relację pomiędzy dziekanem a wydziałem opiszemy w klasie `FacultyAndDean`

In [ ]:
data class FacultyAndDean (

Ponieważ mamy relację 1-1 to jej kierunek nie ma znaczenia - w tym przykładzie mamy pole reprezentujące wydział - `facultyName` w klasie `Dean`, więc powiążemy dziekana z wydziałem. Pole `facultyName` w klasie `Dean` jest tzw. **kluczem obcym**.

In [ ]:
data class FacultyAndDean (
    @Embedded val faculty: Faculty,

W pierwszym kroku dodajemy pole `Faculty` i oznaczamy je jako `@Embedded` - adnotacja ta informuje kompilator że pole `faculty` jest bytem (`@Entity`) bazy `ROOM`, a co za tym idzie, może uzyskać do niego dostęp bezpośrednio przez wygenerowanie zapytania `SQL`.

Drugim polem będzie instancja klasy `Dean` - już bez adnotacji.

In [ ]:
public class FacultyAndDean {
    @Embedded
    public Faculty faculty;

    public Dean dean;
}

Po wybraniu odpowiedniego wydziału (przez automatycznie wygenerowanie zapytanie `SQL`) do pola `dean` zostanie zwrócony przydzielony dziekan.

Pozostaje nam określić relację pomiędzy tymi dwiema tabelami

In [ ]:
data class FacultyAndDean (
    @Embedded val faculty: Faculty,

    @Relation(
        parentColumn = "facultyName", // Faculty
        entityColumn = "facultyName"  // Student
    )

    val dean: Dean
    )

Czyli porównujemy, czy pole `facultyName` klasy `Faculty` oraz `facultyName` klasy `Dean`. Czyli klasa `FacultyAndDean` umożliwia automatyczne wykonanie `JOIN` i zwrócenie odpowiednich elementów z obu tabel.

Przejdźmy do utworzenia `FacultyDao`

In [ ]:
@Dao
interface FacultyDao {

Dodajmy kilka metod
- dodanie nowego dziekana

In [ ]:
    @Insert(onConflict = OnConflictStrategy.IGNORE)
    suspend fun insertDean(dean: Dean)

- dodanie nowego wydziału

In [ ]:
    @Insert(onConflict = OnConflictStrategy.IGNORE)
    suspend fun insertFaculty(faculty: Faculty)

- metoda zwracająca wydział oraz przypisany do niego dziekan. Ponieważ pracujemy tutaj na dwóch tabelach, chcemy zapewnić bezpieczeństwo zec względu na wątki - w tym celu używamy adnotacji `@Transaction`. W tej metodzie odwołujemy się bezpośrednio do tabeli `faculty`, jednak będziemy również przeszukiwać tabelę `dean`, więc nie chcemy aby w tym samym czasie inny wątek tą tabelę np. modyfikował.

In [ ]:
    @Transaction
    @Query("SELECT * FROM faculty WHERE facultyName = :facultyName")
    suspend fun getFacultyAndDean(facultyName: String): FacultyAndDean

Dodajmy klasę `FacultyRoomDatabase`

In [ ]:
@Database(
    entities = [
        Faculty::class,
        Dean::class,
    ],
    version = 1,
    exportSchema = false
)
abstract class FacultyRoomDatabase : RoomDatabase() {
    abstract val facultyDao: FacultyDao

    companion object{
        @Volatile
        private var INSTANCE: FacultyRoomDatabase? = null

        fun getInstance(context: Context): FacultyRoomDatabase{
            synchronized(this){
                return  INSTANCE ?: Room.databaseBuilder(
                    context.applicationContext,
                    FacultyRoomDatabase::class.java,
                    "kotlin_faculty_db"
                ).build().also {
                    INSTANCE = it
                }
            }
        }
    }
}

Zwróćmy uwagę, że w polu `entities` musimy podać **wszystkie** klasy oznaczone jako `@Entity`. Również tutaj mamy 'akademicką' implementację singletona.

Oraz w głównej aktywności zainicjujmy bazę i dodajmy kilka elementów

In [ ]:
override fun onCreate(savedInstanceState: Bundle?) {
        super.onCreate(savedInstanceState)
        setContentView(R.layout.activity_main)

        val dao = FacultyRoomDatabase.getInstance(this).facultyDao

        val faculties = listOf(
            Faculty("Physics and Astronomy"),
            Faculty("Computer Science"),
            Faculty("Psychology")
        )

        val deans = listOf(
            Dean("Marek P", "Computer Science"),
            Dean("Michal P", "Psychology"),
            Dean("Arek P", "Physics and Astronomy"),
        )

        )

        lifecycleScope.launch{
            faculties.forEach { dao.insertFaculty(it) }
            deans.forEach { dao.insertDean(it) }
            
            val myFaculty: FacultyAndDean = dao.getFacultyAndDean("Physics and Astronomy")
            val t1 = findViewById<TextView>(R.id.textView1)
            val t2 = findViewById<TextView>(R.id.textView2)
            t1.text = myFaculty.faculty.facultyName
            t2.text = myFaculty.dean.deanName
        }
    }

Zbudujmy aplikację i wykorzystajmy `App Inspector` aby przjerzeć bazę

<img src="https://media3.giphy.com/media/qe7jBhllmpgnYIWqO2/giphy.gif?cid=790b76115d6f610efc818124664ae35fd2920fe965443eba&rid=giphy.gif&ct=g" width="400" />

Następnie spróbujmy wykorzystać metodę `getFacultyAndDean`

<img src="https://media1.giphy.com/media/BKAimOcA8fEqotVXIA/giphy.gif?cid=790b7611f9b14783bd71567e24ad8b3ffd7d859e1f91f8f4&rid=giphy.gif&ct=g" width="200" />

### **Relacja 1-N**

Przjedźmy do relacji 1-wiele, czyli do jednego elementu z tabeli może być przypisanych wiele elementów z innej tabeli. Do naszej bazy dodamy tabelę reprezentującą studenta - mamy wielu studentów powiązanych z jednym wydziałem (na potrzeby przykładu przyjmiemy że student może być powiązany tylko z jednym wydziałem)

W tej relacji już ma znaczenie klucz której klasy jest kluczem obcym. Musimy do klasy `Student` dołączyć `facyltuName` jako klucz obcy. Dodajmy klasę `@Entity` reprezentującą studenta

In [ ]:
@Entity
data class Student (
    @PrimaryKey(autoGenerate = false)
    val studentName: String,
    val indexNumber: Int,
    val facultyName: String
        )

Następnie określmy relację pomiędzy klasą `Student` a klasą `Faculty`, różnicą do poprzedniego przykładu jest to, że zastosujemy listę do przechowania wszystkich studentów powiązanych z danym wydziałem

In [ ]:
data class FacultyWithStudents (
    @Embedded val faculty: Faculty,

    @Relation(
        parentColumn = "facultyName",
        entityColumn = "facultyName"
    )

    val students: List<Student>
        )

Dodajmy kilka metod do `FacultyDao`

In [ ]:
    @Insert(onConflict = OnConflictStrategy.IGNORE)
    suspend fun insertStudent(student: Student)

    @Transaction
    @Query("SELECT * FROM student WHERE facultyName = :facultyName")
    suspend fun getFacultyWithStudents(facultyName: String): List<FacultyWithStudents>
}

Umożliwiamy dodanie nowego studenta, oraz zwrócenie z bazy wszystkich studentów z określonym wydziałem. Musimy również dodać do listry `entities` w bazie danych klasę `Student`

In [ ]:
@Database(
    entities = [
        Faculty::class,
        Dean::class,
        Student::class,
    ],
    version = 1,
    exportSchema = false
)
abstract class FacultyRoomDatabase : RoomDatabase() {

### **Relacja M-N**

Ostatnią relacją której się przyjrzymy będzie relacja wiele-wiele. Czyli wielu elementom jednej tabeli może być przypisanych wiele elementów innej tabeli. Dodamy `@Entity` reprezentujący wykład - na każdy wykład może uczęszczać wielu studentów jak również jeden student może chodzić na wiele wykładów. Rozpocznijmy od dodania klasy reprezentującej wykład

In [ ]:
@Entity
data class Lecture (
    @PrimaryKey(autoGenerate = false)
    val lectureName: String
        )

Tutaj nie będziemy dodawać kluczy obcych bezpośrednio do klas - musimy utworzyć nową klasę do której dodamy klucze z obu tabel jako klucze obce. Dodajmy nową klasę `StudentLectureCrossRef`

In [ ]:
@Entity(primaryKeys = ["studentName", "lectureName"])
data class StudentLectureCrossRef (

Tutaj w adnotacji `@Entity` dodajemy informację o kluczach głównych obu tabel dla których tworzymy `CrossRef`. Umoliwi nam to wyciąganie z bazy listy wszystkich studentów uczęszczających na jeden konkretny wykład, jak i również listę wszystkich wykładów na które jest zapisany jeden konkretny student .

In [ ]:
@Entity(primaryKeys = ["studentName", "lectureName"])
data class StudentLectureCrossRef (
    val studentName: String,
    val lectureName: String
        )

Jeżeli tworzenie relacji wymaga indeksowania możemy posłużyć się adnotacją `@ColumnInfo(index = true)` - nie każda relacja wiele-wiele wymaga indeksowania. Musimy teraz zdefiniować dwie relacje 1-wiele w których klasa `StudentLectureCrossRef` będzie wykorzystana jako klasa pomocnicza.

In [ ]:
data class LectureWithStudents (
    @Embedded val lecture: Lecture,

    @Relation(
        parentColumn = "lectureName",
        entityColumn = "studentName",
        associateBy = Junction(StudentLectureCrossRef::class)
    )

    val students: List<Student>
        )

In [ ]:
data class StudentWithLectures (
    @Embedded val student: Student,

    @Relation(
        parentColumn = "studentName",
        entityColumn = "lectureName",
        associateBy = Junction(StudentLectureCrossRef::class)
    )

    val lectures: List<Lecture>
    )

Jak widzimy klasy te różnią się niewiele od określania relacji 1-wiele - różnicą jest zastosowanie adnotacji `@Junction`. Ponieważ porównanie pól `studentName` i `lectureName` nie jest możliwe, wskazujemy bazie `ROOM` w jaki sposób te dwa pola są ze sobą powiązane za pomocą `associateBy`. Tutaj wymagana jest adnotacja `@Junction` (deklaruje *junction* który zostanie użyty do wykonania `JOIN`)

Dodajmy kilka metod do `FacultyDao`

In [ ]:
@Dao
interface FacultyDao {

    @Insert(onConflict = OnConflictStrategy.IGNORE)
    fun insertFaculty(faculty: Faculty)

    @Insert(onConflict = OnConflictStrategy.IGNORE)
    fun insertDean(dean: Dean)

    @Insert(onConflict = OnConflictStrategy.IGNORE)
    fun insertStudent(student: Student)

    @Insert(onConflict = OnConflictStrategy.IGNORE)
    fun insertStudentLectureCrossRef(crossRef: StudentLectureCrossRef)

    @Insert(onConflict = OnConflictStrategy.IGNORE)
    fun insertLecture(lecture: Lecture)

    @Transaction
    @Query("SELECT * FROM faculty WHERE facultyName = :facultyName")
    fun getFacultyAndDean(facultyName: String): FacultyAndDean

    @Transaction
    @Query("SELECT * FROM student WHERE facultyName = :facultyName")
    fun getFacultyWithStudents(facultyName: String): List<FacultyWithStudents>

Oraz zmodyfikujmy `entities` w naszej bazie

In [ ]:
@Database(
    entities = [
        Faculty::class,
        Dean::class,
        Student::class,
        Lecture::class,
        StudentLectureCrossRef::class
    ],
    version = 1,
    exportSchema = false
)
abstract class FacultyRoomDatabase : RoomDatabase() {

Dodajmy trochę danych testowych o zobaczmy bazę w `App Inspector`

In [ ]:
override fun onCreate(savedInstanceState: Bundle?) {
        super.onCreate(savedInstanceState)
        setContentView(R.layout.activity_main)

        val dao = FacultyRoomDatabase.getInstance(this).facultyDao

        val faculties = listOf(
            Faculty("Physics and Astronomy"),
            Faculty("Computer Science"),
            Faculty("Psychology")
        )

        val lectures = listOf(
            Lecture("PUM"),
            Lecture("C programming"),
            Lecture("Basic Psychology"),
            Lecture("Fundamental Physics")
        )

        val students = listOf(
            Student("Raf Lew", 1, "Physics and Astronomy"),
            Student("Lew Raf", 2, "Computer Science"),
            Student("R Lew", 3, "Physics and Astronomy"),
            Student("Raf L", 4, "Computer Science"),
            Student("Rafal Lew", 5, "Psychology"),
        )

        val deans = listOf(
            Dean("Marek P", "Computer Science"),
            Dean("Michal P", "Psychology"),
            Dean("Arek P", "Physics and Astronomy"),
        )

        val studentsLectureRelations = listOf(
            StudentLectureCrossRef("Raf Lew", "PUM"),
            StudentLectureCrossRef("Raf Lew", "C Programming"),
            StudentLectureCrossRef("Raf Lew", "Fundamental Physics"),
            StudentLectureCrossRef("R Lew", "PUM"),
            StudentLectureCrossRef("R Lew", "Basic Psychology"),
            StudentLectureCrossRef("Lew Raf", "PUM"),
            StudentLectureCrossRef("Lew Raf", "Fundamental Physics"),
            StudentLectureCrossRef("Raf L", "PUM")
        )

        lifecycleScope.launch{
            faculties.forEach { dao.insertFaculty(it) }
            deans.forEach { dao.insertDean(it) }
            students.forEach { dao.insertStudent(it) }
            lectures.forEach { dao.insertLecture(it) }
            studentsLectureRelations.forEach { dao.insertStudentLectureCrossRef(it) }
            val myFaculty: FacultyAndDean = dao.getFacultyAndDean("Physics and Astronomy")
            val t1 = findViewById<TextView>(R.id.textView1)
            val t2 = findViewById<TextView>(R.id.textView2)
            t1.text = myFaculty.faculty.facultyName
            t2.text = myFaculty.dean.deanName
        }
    }

<img src="https://media4.giphy.com/media/ydNTPPeaGmYTSySrrc/giphy.gif?cid=790b7611dc0a9473e080b7798c1208ad1c8de9b2ad91c849&rid=giphy.gif&ct=g" width="400" />